In [ ]:
import pandas as pd
import numpy as np
diamonds = pd.read_csv("/content/diamonds.csv")
diamonds.head()

### Step 1: pre-start

In [ ]:
#remove the indixing column
diamonds = diamonds.drop('Unnamed: 0', axis=1)
diamonds.columns

### Step 2: See the data info:

In [ ]:
#what are the attributes?
diamonds.info()

In [ ]:
#see more info about the numeric attributes
diamonds.describe()

In [ ]:
#how many rows have 0 values for x or y or z:
diamonds[(diamonds['y'] == 0) | (diamonds['x'] == 0) | (diamonds['z'] == 0)].shape

so we only have 20 rows from +50K rows, lets drop it

In [ ]:
diamonds = diamonds[(diamonds['y'] != 0) & (diamonds['x'] != 0) & (diamonds['z'] != 0)]
#reindixing the data
diamonds.index = np.arange(0,53920)

diamonds.shape
#53940-20=53920 .... OK

In [ ]:
diamonds.isna().count()
# very doog

In [ ]:
#lets look now at the linear relation strength between different attributes
diamonds.corr()

Let's take a ghraphical look at the data:

In [ ]:
diamonds.hist(bins=50, figsize=(20,15))

Notice that prices varies so musch, we may need to standarize them

In [ ]:
#a nother look at the correlation:
import seaborn as sns
sns.heatmap(data = diamonds.corr(), square = True, annot = True, cmap = 'BuPu')

In [ ]:
from pandas.plotting import scatter_matrix
attributes = ["price", "table", "depth"]
scatter_matrix(diamonds[attributes], figsize=(20, 20))

In [ ]:
from pandas.plotting import scatter_matrix
attributes = ["price", "x", "y", "z"]
scatter_matrix(diamonds[attributes], figsize=(20, 20))

In [ ]:
#a deeper look at the price, carat diagram
diamonds.plot(kind="scatter", x="carat", y="price", alpha=0.1)

In [ ]:
sns.factorplot( data = diamonds, x = 'cut', kind = 'count', aspect =2)

In [ ]:
sns.factorplot( data = diamonds, x = 'clarity', kind = 'count', aspect =2.5)

In [ ]:
sns.factorplot( data = diamonds, x = 'color', kind = 'count', aspect =2.5)

In [ ]:
sns.factorplot( data = diamonds, x = 'cut', y = 'price', kind = 'box', aspect =2.5)

In [ ]:
sns.factorplot(x = 'clarity', y = 'price', data = diamonds, kind = 'box', aspect = 2.5)

Untill now we've seen the numeric attributs, let see the Text attributes: 'cut', 'color', 'clarity':

In [ ]:
diamonds['cut'].unique()
# we have 5 features, related to each other so that we can tidy them like:
# ['Fair' , 'Good', 'Very Good','Premium', 'Ideal']
# we should use ordinal encoding

In [ ]:
# let's see how many times each feature repeated?
diamonds['cut'].value_counts()

In [ ]:
diamonds['clarity'].unique()
# we have 8 features, related to each other so that we can tidy them like:
# ['Fair' , 'Good', 'Very Good','Premium', 'Ideal']
# we should use ordinal encoding

In [ ]:
# let's see how many times each feature repeated?
diamonds['clarity'].value_counts()

In [ ]:
diamonds['color'].unique()
# we have 7 features, does not related to each other so we should use OneHot encoding

In [ ]:
diamonds.head()

# Convert the Text attibutes

In [ ]:
# 'cut'
diamonds_cut = diamonds[["cut"]]
from sklearn.preprocessing import OrdinalEncoder
cut_ordinal_encoder = OrdinalEncoder()
diamonds_cut_encoded = cut_ordinal_encoder.fit_transform(diamonds_cut)


#You  can  get  the  list  of  categories  using  the  categories_  instance  variable.  
print(cut_ordinal_encoder.categories)
diamonds_cut_encoded

In [ ]:
# 'color'
diamonds_color = diamonds[["color"]]
from sklearn.preprocessing import OrdinalEncoder
color_ordinal_encoder = OrdinalEncoder()
diamonds_color_encoded = color_ordinal_encoder.fit_transform(diamonds_color)

print(color_ordinal_encoder.categories_)

In [ ]:
# 'clarity'
diamonds_clarity = diamonds[["clarity"]]
from sklearn.preprocessing import OneHotEncoder
clarity_OneHot_encoder = OneHotEncoder()
diamonds_clarity_encoded = clarity_OneHot_encoder.fit_transform(diamonds_clarity)

print(clarity_OneHot_encoder.categories_)

In [ ]:
diamonds.head()

In [ ]:
#replace the columns
diamonds_prepared = diamonds.copy()
diamonds_prepared

In [ ]:
diamonds_prepared.drop(['cut', 'color', 'clarity'], axis=1, inplace=True)

diamonds_num = diamonds_prepared.copy()

diamonds_prepared['size']=diamonds_prepared['x']*diamonds_prepared['y']*diamonds_prepared['z']
diamonds_prepared['area']=diamonds_prepared['x']*diamonds_prepared['y']

arr_diamonds_clarity_encoded = diamonds_clarity_encoded.toarray()

diamonds_cut_pd = pd.DataFrame(diamonds_cut_encoded)
diamonds_color_pd = pd.DataFrame(diamonds_color_encoded)
diamonds_clarity_pd = pd.DataFrame(arr_diamonds_clarity_encoded)

In [ ]:
print(diamonds_prepared)
print(diamonds_cut_pd)
#diamonds_prepared['cut'] = diamonds_cut_pd
diamonds_prepared.describe()

In [ ]:
diamonds_prepared['color'] = diamonds_color_pd

In [ ]:
diamonds_prepared= diamonds_prepared.join(diamonds_clarity_pd)
diamonds_prepared.head()

In [ ]:
#a nother look at the correlation:
corr = diamonds_prepared.corr()
corr['price'].sort_values(ascending=False)

notice the stong relation between size and 
Dealing with the outliers:

In [ ]:
import seaborn as sns
sns.boxplot(x="price", y='cut', data=diamonds)

In [ ]:
import seaborn as sns
sns.boxplot(x="price", y="clarity", data=diamonds)

In [ ]:
import seaborn as sns
sns.boxplot(x="price", y="color", data=diamonds)

In [ ]:
import matplotlib.pyplot as plt
labels = diamonds.clarity.unique().tolist()
sizes = diamonds.clarity.value_counts().tolist()
colors = ['#006400', '#E40E00', '#A00994', '#613205', '#FFED0D', '#16F5A7','#ff9999','#66b3ff']
explode = (0.1, 0.0, 0.1, 0, 0.1, 0, 0.1,0)
plt.pie(sizes, explode=explode, labels=labels, colors=colors,autopct='%1.1f%%', shadow=True, startangle=0)
plt.axis('equal')
plt.plot()
fig=plt.gcf()
plt.show()

# To Do

### better relation fo size attribute?...

### who to tide ordinal encoder?...

### Who to remove outliers?....

# Split data for training

In [ ]:
diamonds_prepared_num = diamonds_prepared.drop("price", axis=1) # we will use this DataFrame later

from sklearn.model_selection import train_test_split
train_set, test_set= train_test_split(diamonds_prepared, test_size=0.2, random_state=42)

final_train_labels = train_set['price'].copy()
final_train_set = train_set.drop(['price'], axis=1)

final_test_labels = test_set['price'].copy()
final_test_set = test_set.drop(['price'], axis=1)

train_set.head()

In [ ]:
final_train_set.info()

In [ ]:
final_train_set.isna().count()

In [ ]:
print(final_train_set.shape)
final_train_set.describe()

In [ ]:
final_train_set[final_train_set['y'] > 40]

In [ ]:
print(final_train_labels.shape)
final_train_labels.head()

## LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

lin_reg = LinearRegression()
lin_reg.fit(final_train_set, final_train_labels)
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
diamonds_predictions = lin_reg.predict(final_test_set)
lin_mse = mean_squared_error(final_test_labels, diamonds_predictions)
lin_rmse = np.sqrt(lin_mse)

print("MSR:        % 0.2f " %  lin_rmse)
print("mean:      % 0.2f " %  final_test_labels.mean())
print("min:         % 0.2f " % final_test_labels.min())
print("max:        % 0.2f " %  final_test_labels.max())

## DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(final_train_set, final_train_labels)
diamonds_predictions = tree_reg.predict(final_test_set)
tree_mse = mean_squared_error(final_test_labels, diamonds_predictions)
tree_rmse = np.sqrt(tree_mse)

print("MSR:        % 0.2f " %  tree_rmse)

## cross_val_score

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, final_train_set, final_train_set, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
  print("Scores: ", scores)
  print("Mean: ", scores.mean())
  print("Standard deviation", scores.std())

In [ ]:
display_scores(tree_rmse_scores)

In [ ]:
from sklearn.model_selection import cross_val_score
lin_scores = cross_val_score(lin_reg,final_train_set, final_train_labels, scoring="neg_mean_squared_error",cv=10)
lin_rese_scores = np.sqrt(-lin_scores)

display_scores(lin_rese_scores)

# Fine you model

In [119]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = [
              {'n_estimators':[3,10,30], 'max_features': [2,4,6,8]}, 
              {'bootstrap': [False], 'n_estimators': [3,10], 'max_features':[2,3,4]},
              ]
forest_reg = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(final_train_set, final_train_labels)

diamonds_predictions = grid_search.predict(final_test_set)
grid_mse = mean_squared_error(final_test_labels, diamonds_predictions)
grid_rmse = np.sqrt(grid_mse)

print("MSR:        % 0.2f " %  grid_rmse)

MSR:         535.57 


In [120]:
grid_search.best_params_

grid_search.best_estimator_

cvers = grid_search.cv_results_

for mean_srore, params in zip(cvers["mean_test_score"], cvers["params"]):
  print(np.sqrt(-mean_srore), params)

817.7274559933992 {'max_features': 2, 'n_estimators': 3}
681.0203712319792 {'max_features': 2, 'n_estimators': 10}
647.5665668103007 {'max_features': 2, 'n_estimators': 30}
711.942002470978 {'max_features': 4, 'n_estimators': 3}
622.3603851622819 {'max_features': 4, 'n_estimators': 10}
591.7545404432731 {'max_features': 4, 'n_estimators': 30}
682.0229994413908 {'max_features': 6, 'n_estimators': 3}
603.2848391254541 {'max_features': 6, 'n_estimators': 10}
575.5980353880889 {'max_features': 6, 'n_estimators': 30}
657.4983066130238 {'max_features': 8, 'n_estimators': 3}
590.5590097728265 {'max_features': 8, 'n_estimators': 10}
566.9279597239292 {'max_features': 8, 'n_estimators': 30}
777.2454516462395 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
658.4532468477772 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
724.4510332433148 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
631.340312089648 {'bootstrap': False, 'max_features': 3, 'n_estimators': 10}


# Analyze the model

In [121]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

"""
1-get the predictors and the labels from your test set
"""

# CODE HERE
final_predictions = grid_search.predict(final_test_set)

"""2-run your full_pipeline to transform the data"""

# CODE HERE
final_mse = mean_squared_error(final_test_labels, final_predictions)

"""3-evaluate the final model on the test set"""

# CODE HERE
final_rmse = np.sqrt(final_mse)
final_rmse

535.5652963298558